# 台灣選舉模擬器 - 2018

[2014台北市模擬器](https://ntuaha.github.io/VoteSimulator/)


> 你覺得台北市特定候選人贏了嗎? 別急! 先看看數據吧，退潮的時候才知道誰沒穿泳褲 ~ 2014 aha


> 佛系丁丁, Togi姚姚 v.s. 柯粉 - 2018 aha


In [160]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",2000)

## 緣起
> 每次選舉我們都會很好奇，到底民調與最後選舉結果是不是有相關性．但很少看見有人真的去根據民調與其他的資料搭配之後，將預估得票數計算做成報告．因此希望透過這次的台北市長選舉相關資料，能夠讓這個問題得到初步的解決．我真正是害怕這個連結....[沈默的多數](https://www.youtube.com/watch?v=t8OtsCuXyaw&feature=youtu.be) 2014 aha


但是這次選舉實在太撲朔離迷了，[丁丁個人才](https://www.youtube.com/watch?v=xnR4UUA2Dac)，[Togi 一～三世](https://news.tvbs.com.tw/local/961983)，[柯P的兩岸一家親](https://udn.com/news/story/6656/3304602)，分別都受到媒體的持續討論，看起來很熱，但嚴格來說台北市的選舉還是算相對平靜的，因為至少不論誰選上，中國民國都不會滅亡．不過相對的今年的模擬也就更加的困難了

## 資料源

項目|說明
:-:|:-:
[2014 主計處各縣市人口統計](http://www.dgbas.gov.tw/ct.asp?xItem=15408&CtNode=4594&mp=1)|每年都會有一份統計數據放在主計處網站，這次為了快，手動自己解出來
[2008 TVBS 總統大選後兩天民調](http://www1.tvbs.com.tw/FILE_DB/DL_DB/even/200804/even-20080402101905.pdf)|用當時強調改變的氣氛來詮釋這是台北市各年齡層的投票比例
[2014/11/15 趨勢.民意調查](http://www.polls.com.tw/webc/html/news/show.aspx?root=1&num=166&page=1)|法律規定最後一次的民意調查，根據過去經驗，越接近越準    
[TVBS民調中心](http://www.tvbs.com.tw/poll-center)|用來追蹤各個候選人民調狀態
[2014市長得票數量](http://db.cec.gov.tw/histMain.jsp?voteSel=20141101B1)|得知最後結果
[20180724 TVBS 電訪民調](https://cc.tvbs.com.tw/portal/file/poll_center/2018/20180724/1f034fa1ba6e4ca57cad6ee57d0a48ac.pdf)| 20180724的調查結果
    

## 探討問題
本研究主要是針對以下幾個變數做調整

### 原始資料
1. 台北市 2014 最終得票
2. 台北市 2014 年各年齡投票人數
2. 各方2014最後一次的民調 － 2014/11/15
3. 最新的民調 － 2014/11/15

### 預測資料

1. 台北市 2018 年可投票人數
2. 各年齡層真的會去投票比率
3. 計算公式

```
 預估總投票數 = 總和 2010各年齡層投票率*2018台北市各年齡人數   
 丁丁得票數 = 2016 總統大選藍軍台北得票率 * 預估總得票數 
 姚文智得票數 = 2008 總統大選綠軍台北得票率 * 預估總得票數 
 柯文哲得票數 = 預估總投票數 - 丁丁得票數 - 姚文智得票數 
``` 

3. 初估預測
4. 模型微調

## 2014

In [215]:
def get_population(year='2014',out='./tp_2014.xlsx'):
    p2 = pd.read_excel('./data/人口數量.xlsx',sheet_name=year)
    p2.columns = p2.iloc[1,]
    p2 = p2.iloc[6:-7,:24]
    p2.reset_index(drop=True,inplace=True)
    city = p2.iloc[p2.index % 3 == 0,0]
    city = city.str.replace(" ","").str.replace("\"","")
    city = np.vectorize(lambda x: x[:3])(city.str)
    man = p2.iloc[p2.index % 3 == 1,:]
    man["區 域 別\nLocality"] = city
    woman = p2.iloc[p2.index % 3 == 2,:]
    woman["區 域 別\nLocality"] = city
    woman.drop(columns='性別\nSex',inplace=True)
    woman.rename(columns={'區 域 別\nLocality':'city'},inplace=True)
    man.drop(columns='性別\nSex',inplace=True)
    man.rename(columns={'區 域 別\nLocality':'city'},inplace=True)
    woman['80歲以上'] = np.sum(woman.iloc[:,-5:],axis=1)
    woman2 = woman.iloc[[1,5],[0,6,7,8,9,10,11,12,13,14,15,16,17,-1]]
    man['80歲以上'] = np.sum(man.iloc[:,-5:],axis=1)
    man2 = man.iloc[[1,5],[0,6,7,8,9,10,11,12,13,14,15,16,17,-1]]
    source = pd.DataFrame({'age':vote_rate_2016[(vote_rate_2016['city']=="臺北市") & (vote_rate_2016['gender']=="male")]['age']})
    source['man'] = man2.iloc[0,1:].astype(np.float).values
    source['woman'] = woman2.iloc[0,1:].astype(np.float).values
    source['population'] = source['man']+source['woman']
    source.to_excel(out)
    return source


In [216]:
get_population()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

,age,man,woman,population
1,20,78552.0,73103.0,151655.0
3,25,78447.0,80882.0,159329.0
5,30,105245.0,118719.0,223964.0
7,35,107951.0,123852.0,231803.0
9,40,96222.0,111729.0,207951.0
11,45,96535.0,112049.0,208584.0
13,50,98411.0,112322.0,210733.0
15,55,96092.0,110635.0,206727.0
17,60,87691.0,100472.0,188163.0
19,65,55867.0,64949.0,120816.0


## 2018



- [2018各區域人口數量](https://view.officeapps.live.com/op/view.aspx?src=https://www.moi.gov.tw/files/site_stuff/321/1/month/m1-06.xls)

In [30]:
population = pd.read_excel('./data/人口數量.xlsx',sheet_name='2018')

In [31]:
population.columns = population.iloc[1,]

In [32]:
#population.columns = np.arange(len(population.columns))

In [33]:
population = population.iloc[6:,:24]

In [34]:
population.columns[0]

'區 域 別\nLocality'

In [36]:
#population['區 域 別\nLocality']

In [39]:
population = population[~population['區 域 別\nLocality'].isna() & ~population['性別\nSex'].isna() ]
population.reset_index(drop=True,inplace=True)

In [42]:
population['區 域 別\nLocality'] = population['區 域 別\nLocality'].str.replace(" ","").str.replace("\"","")

In [44]:
population['區 域 別\nLocality'] = np.vectorize(lambda x: x[:3])(population['區 域 別\nLocality'].str)

In [46]:
population.drop(columns='性別\nSex',inplace=True)

In [53]:
population.columns

Index(['區 域 別\nLocality', '總人口數', '0~4歲', '5~9歲', '10~14歲', '15~19歲', '20~24歲',
       '25~29歲', '30~34歲', '35~39歲', '40~44歲', '45~49歲', '50~54歲', '55~59歲',
       '60~64歲', '65~69歲', '70~74歲', '75~79歲', '80~84歲', '85~89歲', '90~94歲',
       '95~99歲', '100歲以上'],
      dtype='object', name=1)

- [2016總統大選投票資訊](http://db.cec.gov.tw/histQuery.jsp?voteCode=20160101P1A1&qryType=prof&prvCode=00)
- [2012總統大選投票資訊](http://db.cec.gov.tw/histQuery.jsp?voteCode=20120101P1A1&qryType=prof&prvCode=00)
- [2014縣市長投票資訊](http://db.cec.gov.tw/histQuery.jsp?voteCode=20141101C1C1&qryType=prof&prvCode=0)
- [2014直轄市長投票資訊](http://db.cec.gov.tw/histQuery.jsp?voteCode=20141101B1B1&qryType=prof&prvCode=0)
- [2010直轄市長投票資訊](http://db.cec.gov.tw/histQuery.jsp?voteCode=20101101C1B1&qryType=prof&prvCode=0)

In [96]:
votes = pd.DataFrame({'city':population.iloc[:,0],'populations':population['總人口數'],'voters':np.sum(population[population.columns[-17:]],axis=1)})

In [97]:
vote_info_2016 = pd.read_excel('./data/2016總統大選投票縣市投票資訊.xls')
vote_info_2012 = pd.read_excel('./data/2012總統大選投票縣市投票資訊.xls')


In [98]:
vote_info_2014 = pd.read_excel('./data/2014直轄市長投票資訊.xls')
vote_info_2014.columns = np.arange(len(vote_info_2014.columns))
vote_info_2014.rename(columns={0:"city",13:"vote_rate_2014"},inplace=True)
vote_info_2014 = vote_info_2014[~vote_info_2014['city'].isna()]

In [99]:
vote_info_2010 = pd.read_excel('./data/2010直轄市長投票資訊.xls')
vote_info_2010.columns = np.arange(len(vote_info_2010.columns))
vote_info_2010.rename(columns={0:"city",13:"vote_rate_2010"},inplace=True)
vote_info_2010 = vote_info_2010[~vote_info_2010['city'].isna()]

In [100]:
vote_info_2016.rename(columns={"地區別":"city","投票率":"vote_rate_2016"},inplace=True)
vote_info_2012.rename(columns={"地區別":"city","投票率":"vote_rate_2012"},inplace=True)

In [101]:
votes = pd.merge(votes, vote_info_2016[['city','vote_rate_2016']], on='city', how='left')
votes = pd.merge(votes, vote_info_2012[['city','vote_rate_2012']], on='city', how='left')
votes = pd.merge(votes, vote_info_2014[['city','vote_rate_2014']], on='city', how='left')
votes = pd.merge(votes, vote_info_2010[['city','vote_rate_2010']], on='city', how='left')

In [111]:
taipei_total_votes = int(votes[votes['city']=='臺北市']['voters'].values[0] * 0.7)

In [112]:
taipei_total_votes

1525263

In [113]:
#HTML("根據簡易的亂分析判斷，應台北市這次投票人數應該也是7成左右，就亂估七成，所以總投票數量應該是 %d"%taipei_total_votes)

In [114]:
#votes['可能總有效票數'] = votes['voters'] * 0.7

- [2016總統直轄市投票](http://db.cec.gov.tw/histQuery.jsp?voteCode=20160101P1A1&qryType=ctks&prvCode=00)，台北市藍軍拿到37.49%
- [2008總統直轄市投票](http://db.cec.gov.tw/histQuery.jsp?voteCode=20080301P1A1&qryType=ctks&prvCode=00)，台北市綠軍拿到36.96%
- [2002直轄市長投票](http://db.cec.gov.tw/histQuery.jsp?voteCode=20021201C1B1&qryType=ctks)，台北市綠軍拿到35.89%

- [20181015 美麗島民調 ](http://www.my-formosa.com/DOC_139053.htm)
- [慘考資料]( https://newrocreport.org/archives/1548?fbclid=IwAR36AkMSYOeOZGl_McZY43z9Acr5Ih_szshpwZqnutgApyqYb6gI_7Zrzhw)

[2016 TVBS 洪秀柱總統大選民調](https://cc.tvbs.com.tw/portal/file/poll_center/2017/20170602/0407012.pdf)

In [118]:
blue_rate = 0.3749
green_rate = 0.3696
P = 1 - blue_rate - green_rate

In [119]:
#P = 1 - 0.3749 - 0.3696

In [120]:
blue_votes = taipei_total_votes * blue_rate
green_votes = taipei_total_votes * green_rate
P_votes = taipei_total_votes * P

In [124]:
print("丁守中 %.0f 姚文智 %.0f, 柯P %.0f"%(blue_votes,green_votes,P_votes))

丁守中 571821 姚文智 563737, 柯P 389705


## 年齡層投票率

- [2016各年齡投票率](https://www.cec.gov.tw/upload/file/2017-02-17/2632faa8-582d-4e31-85ab-8d5f2ba6d9e3/0e44676e544ec6326a5c6b81497720e3.pdf)

In [125]:
vote_rate_2016 = pd.read_csv('./data/2016總統投票率.csv')

In [224]:
p2 = pd.read_excel('./data/人口數量.xlsx',sheet_name='2018')
p2.columns = p2.iloc[1,]
p2 = p2.iloc[6:-7,:24]
p2.reset_index(drop=True,inplace=True)
city = p2.iloc[p2.index % 3 == 0,0]
city = city.str.replace(" ","").str.replace("\"","")
city = np.vectorize(lambda x: x[:3])(city.str)

In [225]:
city

array(['新北市', '臺北市', '桃園市', '臺中市', '臺南市', '高雄市', '臺灣省', '宜蘭縣', '新竹縣',
       '苗栗縣', '彰化縣', '南投縣', '雲林縣', '嘉義縣', '屏東縣', '臺東縣', '花蓮縣', '澎湖縣',
       '基隆市', '新竹市', '嘉義市', '福建省', '金門縣', '連江縣'], dtype=object)

In [226]:
man = p2.iloc[p2.index % 3 == 1,:]
man["區 域 別\nLocality"] = city
woman = p2.iloc[p2.index % 3 == 2,:]
woman["區 域 別\nLocality"] = city
woman.drop(columns='性別\nSex',inplace=True)
woman.rename(columns={'區 域 別\nLocality':'city'},inplace=True)
man.drop(columns='性別\nSex',inplace=True)
man.rename(columns={'區 域 別\nLocality':'city'},inplace=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  error

In [227]:
woman['80歲以上'] = np.sum(woman.iloc[:,-5:],axis=1)
woman2 = woman.iloc[[1,5],[0,6,7,8,9,10,11,12,13,14,15,16,17,-1]]
man['80歲以上'] = np.sum(man.iloc[:,-5:],axis=1)
man2 = man.iloc[[1,5],[0,6,7,8,9,10,11,12,13,14,15,16,17,-1]]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [228]:
woman2

1,city,20~24歲,25~29歲,30~34歲,35~39歲,40~44歲,45~49歲,50~54歲,55~59歲,60~64歲,65~69歲,70~74歲,75~79歲,80歲以上
5,臺北市,70544,75878,96102,125472,114473,107549,106787,107688,104362,92385,53456,43929,60717.0
17,高雄市,87383,89255,94064,119535,117752,111458,110043,111262,105173,88019,48403,39731,45792.0


In [229]:
p2 = woman2.values + man2.values

In [230]:
woman2.iloc[0,1:].astype(np.float).values

array([ 70544.,  75878.,  96102., 125472., 114473., 107549., 106787.,
       107688., 104362.,  92385.,  53456.,  43929.,  60717.])

In [231]:
source = pd.DataFrame({'age':vote_rate_2016[(vote_rate_2016['city']=="臺北市") & (vote_rate_2016['gender']=="male")]['age']})
source['man'] = man2.iloc[0,1:].astype(np.float).values
source['woman'] = woman2.iloc[0,1:].astype(np.float).values
source['rate_man'] = vote_rate_2016[(vote_rate_2016['city']=="臺北市") & (vote_rate_2016['gender']=="male")]['rate'].values
source['rate_woman'] = vote_rate_2016[(vote_rate_2016['city']=="臺北市") & (vote_rate_2016['gender']=="female")]['rate'].values
source['vote_man'] = source['man'] * source['rate_man'] / 100
source['vote_woman'] = source['woman'] * source['rate_woman'] / 100
#source.to_excel('./tp.xlsx')

In [232]:
source

,age,man,woman,rate_man,rate_woman,vote_man,vote_woman
1,20,75112.0,70544.0,63.2,66.5,47470.784,46911.760
3,25,77563.0,75878.0,61.4,63.5,47623.682,48182.530
5,30,85936.0,96102.0,60.5,66.1,51991.280,63523.422
7,35,110995.0,125472.0,63.2,64.5,70148.840,80929.440
9,40,99979.0,114473.0,59.7,63.6,59687.463,72804.828
11,45,92548.0,107549.0,64.6,71.2,59786.008,76574.888
13,50,92256.0,106787.0,62.7,72.9,57844.512,77847.723
15,55,92784.0,107688.0,69.9,73.6,64856.016,79258.368
17,60,90161.0,104362.0,76.2,76.1,68702.682,79419.482
19,65,78113.0,92385.0,78.5,77.7,61318.705,71783.145


In [204]:
source2 = pd.DataFrame({'age':vote_rate_2016[(vote_rate_2016['city']=="高雄市") & (vote_rate_2016['gender']=="male")]['age']})
source2['man'] = man2.iloc[1,1:].astype(np.float).values
source2['woman'] = woman2.iloc[1,1:].astype(np.float).values
source2['rate_man'] = vote_rate_2016[(vote_rate_2016['city']=="高雄市") & (vote_rate_2016['gender']=="male")]['rate'].values
source2['rate_woman'] = vote_rate_2016[(vote_rate_2016['city']=="高雄市") & (vote_rate_2016['gender']=="female")]['rate'].values
source2['vote_man'] = source2['man'] * source2['rate_man'] / 100
source2['vote_woman'] = source2['woman'] * source2['rate_woman'] / 100
source2['vote'] = source2['vote_man'] +source2['vote_woman']
source2['people'] = source2['man'] +source2['woman']
source2.to_excel('./ks.xlsx')

In [126]:
vote_rate_2016

,city,gender,age,rate
0,臺北市,female,20,66.5
1,臺北市,male,20,63.2
2,臺北市,female,25,63.5
3,臺北市,male,25,61.4
4,臺北市,female,30,66.1
5,臺北市,male,30,60.5
6,臺北市,female,35,64.5
7,臺北市,male,35,63.2
8,臺北市,female,40,63.6
9,臺北市,male,40,59.7
